In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow import keras
import os

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
zip_PATH = "/content/drive/MyDrive/train_data.zip"
save_PATH = "/content/drive/MyDrive/mobile_net_v2_1.h5"
load_PATH = '/content/train_data'
!unzip -q "/content/drive/MyDrive/train_data.zip"

In [8]:
img_gen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    validation_split=0.2
)

In [9]:
BATCH_SIZE = 32

train_ds = img_gen.flow_from_directory(directory = load_PATH , batch_size = BATCH_SIZE ,
                                       shuffle = True , class_mode = 'sparse' ,
                                       target_size = (299,299) , subset = 'training' , seed = 42)
val_ds = img_gen.flow_from_directory(directory = load_PATH , batch_size = BATCH_SIZE ,
                                     shuffle = False  , class_mode = 'sparse' ,
                                     target_size = (299,299) , subset = 'validation', seed = 42)

Found 7994 images belonging to 10 classes.
Found 1996 images belonging to 10 classes.


In [10]:
from keras.src.saving.saving_lib import optimizer
from keras.applications.mobilenet_v2 import MobileNetV2

num_classes = 10
base_model_1 = MobileNetV2(weights='imagenet' , include_top = False)
x = base_model_1.output
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(1024, activation='relu')(x)
predictions = keras.layers.Dense(num_classes, activation='softmax')(x)
model_1 = keras.Model(inputs=base_model_1.input, outputs=predictions)
model_1.compile(loss = 'sparse_categorical_crossentropy' ,
                     optimizer = 'adam' , metrics = ['accuracy'])
for layer in model_1.layers[:100] :
  layer.trainable = False
for layer in model_1.layers[100:] :
  layer.trainable = True
check_inc_res = keras.callbacks.ModelCheckpoint(save_PATH, monitor='val_accuracy', save_best_only=False)

9406464/9406464 [==============================] - 1s 0us/step


In [11]:
model_1.fit(train_ds , validation_data = val_ds , epochs = 10 , callbacks=[check_inc_res] )

Epoch 1/10
  9/250 [>.............................] - ETA: 48:54 - loss: 2.2874 - accuracy: 0.2222

KeyboardInterrupt: ignored